In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [3]:
ltc_requests = requests.get("http://publicreporting.ltchomes.net/en-ca/Search_Selection.aspx")
ltc = BeautifulSoup(ltc_requests.text)

In [76]:
# ltc_requests.status_code
# ltc_requests.reason
# print(ltc_requests.request.headers)
# ltc_requests.headers

In [75]:
# webscrape list of ltc home names and links
ordered_lists = ltc.find_all("ol")
ltc_list = ordered_lists[1]
home = ltc_list.find_all("a", {"class":"rsLink"})

651

In [8]:
# create lists of names and links
names = []
links = []
for each in home:
    names.append(each.get('title'))
    links.append(each.get('href'))

In [9]:
# modify links list to full url
full_links = []
for i in range(len(links)):
    full_links.append('http://publicreporting.ltchomes.net/en-ca/' + links[i])

In [12]:
# iterate through each of the links and scrape characteristics of ltc homes
addresses = []
cities_postalcodes = []
LHIN = []
licensee = []
management = []
home_type = []
beds = []
short_stay = []
residents_council = []
family_council = []
accreditation = []
counter = -1
for each in full_links:
    counter += 1
#     print(counter)
    soup = requests.get(each)
    soup = BeautifulSoup(soup.text)
    addresses.append(soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeAddress"}).getText())
    cities_postalcodes.append(soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeCity"}).getText())
    profiles = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeProfile_item_Col1"})
    profile_data = profiles.find_all("div", {"class":"Profilerow_col2"})
    try:    
        LHIN.append(profile_data[0].getText())
        licensee.append(profile_data[3].getText())
        management.append(profile_data[4].getText())
        home_type.append(profile_data[5].getText())
        beds.append(profile_data[6].getText())
        short_stay.append(profile_data[7].getText())
        residents_council.append(profile_data[8].getText())
        family_council.append(profile_data[9].getText())
        accreditation.append(profile_data[10].getText())
    except:
        print(counter) # print out records which resulted in error
        LHIN.append(None)
        licensee.append(None)
        management.append(None)
        home_type.append(None)
        beds.append(None)
        short_stay.append(None)
        residents_council.append(None)
        family_council.append(None)
        accreditation.append(None)

324
508


In [14]:
# print out links to LTC homes for which an error was raised during webscraping
print(full_links[324])
print(full_links[508])

http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=7089
http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=C604


In [15]:
# create a df of Ontario LTC homes
df = pd.DataFrame({'name':names, 
                   'address':addresses, 
                   'city and postal code':cities_postalcodes, 
                   'LHIN':LHIN, 
                   'licensee':licensee, 
                   'management':management, 
                   'home type':home_type, 
                   'beds':beds, 
                   'short stay':short_stay, 
                   'residents council':residents_council, 
                   'family council':family_council, 
                   'accreditation':accreditation})
# df.info()
# df.head(20)

In [89]:
# split the 'city and postal code' column into 2 columns 'city' and 'postal code'
df['city'] = df['city and postal code'].str.split(',').str[0]
df['postal code'] = df['city and postal code'].str.split(',').str[1]

# use regex to extract the number of beds to a different column
df['number of beds'] = df['beds'].str.extract(r'(\d+)', expand=False)

df.head(20)
df.nunique()

name                    642
address                 648
city and postal code    624
LHIN                     14
licensee                378
management               26
home type                 4
beds                    211
short stay                2
residents council         2
family council            2
accreditation             2
city                    268
postal code             621
number of beds          211
dtype: int64

In [35]:
# modify links to access inspection data 
full_links_inspection = [each + '&tab=1' for each in full_links]
    

['http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2872&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=C501&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2861&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=M501&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=3041&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2647&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=M536&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2692&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=0956&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2786&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2930&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2982&tab=1',
 'http://publicreporting.ltchomes.net/en-ca/homeprof

In [87]:
# webscrape inspection data for each ltc home 
frames = []
counter = -1
for each in full_links_inspection:
    inspection_types = []
    inspection_dates = []
#     counter += 1
#     print(counter)
    soup = requests.get(each)
    soup = BeautifulSoup(soup.text)
    name = soup.find("div", {"class":"HomeName"}).getText()
    inspections = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeProfile_item_Col3"})
    types = inspections.find_all("div", {"class":"divInspectionTypeDataCol"})
    number_of_types = len(types)
    for each in types:
        inspection_types.append(each.getText())
    dates = inspections.find_all("div", {"class":"divInspectionDateDataCol"})
    number_of_dates = len(dates)
    for each in dates:
        inspection_dates.append(each.getText())
    df_temp = pd.DataFrame({"name": name, 
                            "inspection_types":inspection_types, 
                            "number_of_types":number_of_types, 
                            "inspection_dates":inspection_dates, 
                            "number_of_dates":number_of_dates})
    frames.append(df_temp)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [88]:
# create a df of inspection data
df2 = pd.concat(frames)
print(df2.nunique())
df2.info()

# display data for first 2 ltc homes
pd.set_option('display.max_rows', None)
df2.head(134)


name                 648
inspection_types      29
number_of_types      102
inspection_dates    2548
number_of_dates      102
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23565 entries, 0 to 25
Data columns (total 5 columns):
name                23565 non-null object
inspection_types    23565 non-null object
number_of_types     23565 non-null int64
inspection_dates    23565 non-null object
number_of_dates     23565 non-null int64
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


,name,inspection_types,number_of_types,inspection_dates,number_of_dates
0,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Complaints Inspection,67,"Jan 17, 2020",67
1,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Complaints Inspection,67,"Nov 27, 2019",67
2,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Critical Incident Inspection,67,"Nov 27, 2019",67
3,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Critical Incident Inspection,67,"Oct 28, 2019",67
4,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Critical Incident Inspection,67,"May 01, 2019",67
5,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Complaints Inspection,67,"Mar 28, 2019",67
6,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Critical Incident Inspection,67,"Mar 06, 2019",67
7,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Resident Quality Inspection,67,"Aug 02, 2018",67
8,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Critical Incident Inspection,67,"Feb 02, 2018",67
9,AFTON PARK PLACE LONG TERM CARE COMMUNITY,Resident Quality Inspection,67,"Jun 26, 2017",67


In [84]:
# test scripts by scraping a single ltc site

# scrape profile data
# soup = requests.get(
# "http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2872") 
# soup = BeautifulSoup(soup.text)

# address = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeAddress"}).getText()
# city = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeCity"}).getText()
# profiles = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeProfile_item_Col1"})
# profile_data = profiles.find_all("div", {"class":"Profilerow_col2"})
# profile_data
# LHIN = profile_data[0].getText()
# LHIN

# # scrape inspections data
# soup = requests.get(
# "http://publicreporting.ltchomes.net/en-ca/homeprofile.aspx?Home=2872&tab=1") # add "&tab=1" to url
# soup = BeautifulSoup(soup.text)

# inspections = soup.find("div", {"id":"ctl00_ContentPlaceHolder1_divHomeProfile_item_Col3"})
# inspection_types = inspections.find_all("div", {"class":"divInspectionTypeDataCol"})
# inspection_dates = inspections.find_all("div", {"class":"divInspectionDateDataCol"})

# print(len(inspection_types))
# print(len(inspection_dates))

In [85]:
# a = []
# for each in inspection_types:
#     a.append(each.getText())
# a